# EXPERIMENT 5

### SHIVAM SINGHAL 2K18/CO/340

**AIM:** Write a program to demonstrate the working of decision tree based CART algorithm. Build the
decision tree and classify a new sample using suitable dataset. Compare the performance with
that of ID, C4.5, and CART in terms of accuracy, recall, precision and sensitivity.
Also, state the differences b/w the algorithms.

In [1]:
import pandas as pd
import numpy as np 

In [2]:
df = pd.read_csv('main-bank.csv')

In [3]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 4.1+ MB


In [5]:
df.housing.unique()

array(['yes', 'no'], dtype=object)

In [6]:
df = df.drop(['age' , 'balance' , 'day' , 'month' ,'contact', 'duration' , 'campaign' , 'pdays' , 'previous' ], 1)

In [7]:
df.head()

,job,marital,education,default,housing,loan,poutcome,y
0,management,married,tertiary,no,yes,no,unknown,no
1,technician,single,secondary,no,yes,no,unknown,no
2,entrepreneur,married,secondary,no,yes,yes,unknown,no
3,blue-collar,married,unknown,no,yes,no,unknown,no
4,unknown,single,unknown,no,no,no,unknown,no


In [8]:
# Calculating Gini 

In [9]:
def gini(df , col_name):
    Gini = 1  
    values = df[col_name].unique()  
    for value in values:
        pi = df[col_name].value_counts()[value]/len(df[col_name])  
        Gini -= pi*pi
    return Gini

In [10]:
def avg_gini_index(df , col):
    avg_gini = 0
    atributes = df[col].unique()
    count = df[col].count()
    for atr in atributes: 
        Gini = gini(df[df[col] == atr] , 'y')
        cnt = df[df[col] == atr][col].count()
        Gini *= cnt/count
        avg_gini -= Gini
    return avg_gini

In [11]:
def find_winner(df):
    Avg_Gini = []
    for key in df.keys()[:-1]:
        Avg_Gini.append(avg_gini_index(df , key))
    return df.keys()[:-1][np.argmin(Avg_Gini)]

In [12]:
def buildTree(df,tree=None): 
    Class = df.keys()[-1] 
    node = find_winner(df)
    attValue = np.unique(df[node])
    
    if tree is None:                    
        tree={}
        tree[node] = {}

    for value in attValue:
        
        subtable = df[df[node] == value].drop(node , 1)
        
        total_result = subtable.y.count()
        
        arr = subtable.y.unique()
        
        cnt2 = len(subtable.keys())
        
        if len(arr) == 1:
            tree[node][value] = arr[0]
        elif cnt2 == 1:
            yes_count = subtable[subtable['y'] == 'yes']['y'].count()
            no_count = subtable[subtable['y'] == 'no']['y'].count() 
            if(yes_count > no_count) :
                tree[node][value] = 'yes'
            else :
                tree[node][value] = 'no'
        else:
            tree[node][value] = buildTree(subtable)
                   
    return tree

In [13]:
df.head()

,job,marital,education,default,housing,loan,poutcome,y
0,management,married,tertiary,no,yes,no,unknown,no
1,technician,single,secondary,no,yes,no,unknown,no
2,entrepreneur,married,secondary,no,yes,yes,unknown,no
3,blue-collar,married,unknown,no,yes,no,unknown,no
4,unknown,single,unknown,no,no,no,unknown,no


In [14]:
from sklearn.utils import shuffle
df = shuffle(df , random_state = 1)

In [15]:
df.head()
print(len(df))

45211


In [16]:
#partition into training test data
train_len = len(df)*6//10
train_df = df.iloc[:train_len]
test_df = df.iloc[train_len:]
print(len(train_df))
print(len(test_df))

27126
18085


In [17]:
test_df.head()

,job,marital,education,default,housing,loan,poutcome,y
1447,management,married,tertiary,no,yes,no,unknown,no
25576,retired,married,primary,no,no,no,unknown,no
5622,management,divorced,tertiary,no,no,no,unknown,no
9691,blue-collar,married,unknown,no,no,no,unknown,yes
3401,services,divorced,primary,no,yes,no,unknown,no


In [18]:
decision_tree = buildTree(train_df)

In [19]:
# This function will gives the number of correct result from the test dataset
def check(node, tree , test_df ):
    TP = TN = 0
    if len(test_df) == 0 :
        return 0,0
    for child,value in tree.items():
        if value == 'yes' :
            TP += len(test_df[test_df[node]== child][test_df[test_df[node]== child]['y'] == value])
        elif value == 'no' :
            TN += len(test_df[test_df[node]== child][test_df[test_df[node]== child]['y'] == value])
        else :
            next_node = list(value.keys())[0]
            P,N = check(next_node , tree[child][next_node], test_df[test_df[node]== child])
            TP += P
            TN += N
    return TP,TN

In [20]:
next_node = list(decision_tree.keys())[0]
TP,TN = check(next_node, decision_tree[next_node] , test_df)
total_outcome = len(test_df)

In [21]:
FP = len(test_df[test_df['y']=='yes']) - TP
FN = len(test_df[test_df['y']=='no']) - TN

In [22]:
accuracy= (TP+TN) / (TP+FP+TN+FN) * 100
print(accuracy)

88.0619297760575


#### Precision
Precision is the ratio of the correctly +ve labeled by our program to all +ve labeled.

Precision = TP/(TP+FP)

In [23]:
precision = TP / (TP+FP)
print(precision*100)

15.894039735099339


#### Recall (or Sensitivity)
Recall is the ratio of the correctly +ve labeled by our program to all who are diabetic in reality.

Recall = TP/(TP+FN)

In [24]:
recall = TP / (TP+FN)
print(recall*100)

46.86192468619247


#### F1-score
F1 Score considers both precision and recall.

It is the harmonic mean(average) of the precision and recall.

F1 Score = 2*(Recall * Precision) / (Recall + Precision)

In [25]:
f1_score = 2*(recall * precision) / (recall + precision)
print(f1_score*100)

23.73719533733663


#### Specificity
Specificity is the correctly -ve labeled by the program to all who are healthy in reality.

Specificity = TN/(TN+FP)

In [26]:
specificity = TN / (TN + FP)
print(specificity*100)

89.76278212805158


Name of Algo  | Accuracy | Precision | Recall | F1 Score | Specificity
------------- | -------- | --------- | ------ | -------- | ----------- 
ID3  | 88.26% | 16.08% | 49.34% | 24.26% | 89.80%
C4.5  | 88.30% | 16.27% | 49.93% | 24.54% | 89.82% 
CART  | 88.06% | 15.89% | 46.86% | 23.74% | 89.76%